In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns
from scipy import stats
# import researchpy as rp

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [4]:
data_path = '//ic1files.epfl.ch/D-VET/Projects/ChemLab/04_Processing/Processing/Data/PostTest/post_test.pkl'

with open(data_path , "rb") as f:
    df = pd.read_pickle(data_path)

In [5]:
q_start = 2
q_end_number = 6
q_end_sliders = 8
sep_list = ['gender', 'field', 'year', 'level']
q_list = [2, 3, 4, 5, 6, 7]
missing_answer = 'missing_answer'

In [6]:
def compute_percent(df, value, sep, metric):
    q_no = int(value.split('_')[-1])
    
    if sep == 'gender':
        sep_value = int(value.split('_')[1])
    else:
        sep_value = value.split('_')[1]
        
    n1 = df[(df[0, sep] == sep_value) & (df[q_no, 'cat'] == 'Correct')].shape[0]
    n2 = df[df[0, sep] == sep_value].shape[0]
    
    if n2 == 0:
        p = 0
    else:
        p = n1 / n2 * 100
        
    return '  [' + "{:.2f}".format(p) + '% correct]'

In [7]:
def box_plot(df=df, sep='gender', metric='conf'):
    if metric != 'conf' and metric != 'time':
        return
    
    if metric == 'conf':
        columns = [(0, sep), (metric, 2), (metric, 3), (metric, 4), (metric, 5)]
        melt_values = [f'{metric}_2', f'{metric}_3', f'{metric}_4', f'{metric}_5']
    else:
        columns = [(0, sep), (metric, 2), (metric, 3), (metric, 4), (metric, 5), (metric, 6), (metric, 7)]
        melt_values = [f'{metric}_2', f'{metric}_3', f'{metric}_4', f'{metric}_5', f'{metric}_6', f'{metric}_7']
        
    if sep == 'gender':
        df = df[df[(0, sep)] != 3]
    df = df[df[0, sep] != missing_answer]
    
    df2 = df[columns].copy()
    df2.columns = df2.columns.map('{0[0]}_{0[1]}'.format)
    
    df2 = pd.melt(df2, id_vars=[f'0_{sep}'],
                  value_vars=melt_values, value_name="confidence_value")
    df2['sep'] = df2.apply(lambda row: f'{sep}_' + str(row[f'0_{sep}']) + '_' + row['variable'], axis=1)
    
    # add % of correct people per question
    if metric == 'conf':
        df2['sep'] = df2.apply(lambda row: row['sep'] + compute_percent(df, row['sep'], sep, metric), axis=1)
    
    sns.set_theme(style="ticks")
    f, ax = plt.subplots(figsize=(10, 8))

    # Plot the orbital period with horizontal boxes
    sns.boxplot(x="confidence_value", y="variable", data=df2, hue=f'0_{sep}',
                whis=[0, 100], width=.6, palette="vlag")

    # Add in points to show each observation
    sns.stripplot(x="confidence_value", y="variable", data=df2, hue=f'0_{sep}', dodge=True,
                  size=4, color=".3", linewidth=0)

    # Tweak the visual presentation
    ax.xaxis.grid(True)
    ax.set(ylabel="groups")
    ax.set_title(f'{metric} distribution for {sep} for all questions')
    sns.despine(trim=True, left=True)

In [8]:
box_plot(df, 'gender', 'conf')

KeyError: "[('conf', 5), ('conf', 3), ('conf', 4), ('conf', 2)] not in index"